In [81]:
import pandas as pd
import itertools
df = pd.read_csv('D:/Study-3rd/DSP/OP-heist-project/data/combined/trade_by_region_industry_direction_breakdown(employment).csv',encoding='iso-8859-1')

In [82]:
mapping = {
    '10-12 Food industry etc.': 'C Manufacturing (10-33)',
    '13-15 Textile, clothing and leather industry': 'C Manufacturing (10-33)',
    '16 Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials': 'C Manufacturing (10-33)',
    '17, 18 Paper industry; Printing': 'C Manufacturing (10-33)',
    '19-22 Chemical industry': 'C Manufacturing (10-33)',
    '23 Manufacture of other non-metallic mineral products': 'C Manufacturing (10-33)',
    '24-25 Manufacture of basic metals and fabricated metal products, except machinery and equipment': 'C Manufacturing (10-33)',
    '26, 27 Manufacture of electrical and electronic products': 'C Manufacturing (10-33)',
    '28 Manufacture of machinery and equipment n.e.c.': 'C Manufacturing (10-33)',
    '29, 30 Manufacture of transport equipment': 'C Manufacturing (10-33)',
    '31-33 Manufacture of furniture, other manufacturing; repair and installation of machinery and equipment': 'C Manufacturing (10-33)',
    '681, 68209, 683 Other real estate activities': 'L Real estate activities (68)',
    '68201, 68202 Letting and operation of dwellings': 'L Real estate activities (68)'
}
df['Industry'] = df['Industry'].map(lambda x: mapping.get(x, x))
df1 = df.groupby(['Time', 'Area', 'Industry', 'Direction'])['Trade value'].sum().reset_index()
df2 = df1.groupby(['Time','Area','Direction'])['Trade value'].sum().reset_index()

In [83]:
permutations = pd.merge(df1, df2, left_on=['Time', 'Area', 'Direction'],right_on=['Time', 'Area', 'Direction'], how='left')
permutations.rename(columns = {'Trade value_x':'Trade value (euro)','Trade value_y':'Region total trade value (euro)'},inplace = True)
permutations['Time'] = permutations['Time'].astype(str).str[0:4]
permutations['Area'] = permutations['Area'].astype(str).str[0:4]
permutations = permutations.pivot_table(index = ['Time','Area','Industry'],columns = 'Direction',values = ['Trade value (euro)'])
permutations
permutations.reset_index(inplace=True)
permutations.columns = ['Time', 'Area', 'Industry', 'Export', 'Import']

In [84]:
df_gdp = pd.read_csv('D:/Study-3rd/DSP/OP-heist-project/data/Regional_economic_and_environmental_data/gdp_region.csv',encoding = 'iso-8859-1')
df_gdp['Area'] = df_gdp['Area'].astype(str).str[0:4]
df_gdp['2015'] = df_gdp['2015 At current prices, euro'] * df_gdp['2015 Population']
df_gdp['2016'] = df_gdp['2016 At current prices, euro'] * df_gdp['2016 Population']
df_gdp['2017'] = df_gdp['2017 At current prices, euro'] * df_gdp['2017 Population']
df_gdp['2018'] = df_gdp['2018 At current prices, euro'] * df_gdp['2018 Population']
df_gdp['2019'] = df_gdp['2019 At current prices, euro'] * df_gdp['2019 Population']
df_gdp['2020'] = df_gdp['2020 At current prices, euro'] * df_gdp['2020 Population']
df_gdp = df_gdp.drop(columns=df_gdp.columns[1:13])
df_gdp = df_gdp.melt(id_vars = 'Area',var_name = 'Time', value_name = 'Total GDP')
permutations.reset_index(inplace=True)
permutations_new = pd.merge(permutations, df_gdp, on=['Area', 'Time'], how='left')
industries = permutations_new
regions = permutations_new

In [85]:
industries['Industry trade dependency'] = (industries['Export'] + industries['Import'])/industries['Total GDP']*100
industries['Import-Export imbalance'] = (industries['Import'] - industries['Export'])/(industries['Import'] + industries['Export'])
industries = industries.drop(columns = 'index')
industries['Time'] = industries['Time'].astype(int)
years = industries['Time'].unique()
industries_unique = industries['Industry'].unique()
areas = industries['Area'].unique()

for year, indust, area in itertools.product(list(years), list(industries_unique), list(areas)):
    if year == 2015:
        continue  # Skip the first year

    mask_current = (industries['Time'] == year) & (industries['Industry'] == indust) & (industries['Area'] == area)
    mask_prev = (industries['Time'] == year - 1) & (industries['Industry'] == indust) & (industries['Area'] == area)

    curr_exp = industries[mask_current]['Export'].values[0]
    curr_imp = industries[mask_current]['Import'].values[0]
    prev_exp = industries[mask_prev]['Export'].values[0]
    prev_imp = industries[mask_prev]['Import'].values[0]

    export_growth = (curr_exp - prev_exp) / prev_exp
    import_growth = (curr_imp - prev_imp) / prev_imp

    industries.loc[mask_current, 'Export Growth'] = export_growth
    industries.loc[mask_current, 'Import Growth'] = import_growth

industries = industries.fillna(0)

In [86]:
regions = regions.groupby(['Time','Area']).sum()
regions = regions.drop(columns = ['Industry','Industry trade dependency','Import-Export imbalance','index'])
regions['Total GDP'] = regions['Total GDP']/18
regions['Regional trade dependency'] = (regions['Export'] + regions['Import'])/regions['Total GDP']*100
regions['Regional import-Export imbalance'] = (regions['Import'] - regions['Export'])/(regions['Import'] + regions['Export'])
regions.reset_index(inplace=True)
regions.columns = ['Time', 'Area', 'Regional export', 'Regional import', 'Total GDP', 'Regional trade dependency','Regional import-Export imbalance']
regions['Time'] = regions['Time'].astype(int)
years = regions['Time'].unique()
areas = regions['Area'].unique()
for year, area in itertools.product(list(years), list(areas)):
    if year == 2015:
        continue  # Skip the first year

    mask_current = (regions['Time'] == year) & (regions['Area'] == area)
    mask_prev = (regions['Time'] == year - 1) & (regions['Area'] == area)

    curr_exp = regions[mask_current]['Regional export'].values[0]
    curr_imp = regions[mask_current]['Regional import'].values[0]
    prev_exp = regions[mask_prev]['Regional export'].values[0]
    prev_imp = regions[mask_prev]['Regional import'].values[0]

    export_growth = (curr_exp - prev_exp) / prev_exp
    import_growth = (curr_imp - prev_imp) / prev_imp

    regions.loc[mask_current, 'Regional export Growth'] = export_growth
    regions.loc[mask_current, 'Regional import Growth'] = import_growth
regions = regions.fillna(0)

In [87]:
overall = pd.merge(industries,regions,left_on=['Time','Area'],right_on=['Time','Area'],how='left')
overall.rename(columns={'Total GDP_y':'Total GDP','Export':'Industry export','Import':'Industry import','Import-Export imbalance':'Industry import-Export imbalance','Export Growth':'Industry export Growth','Import Growth':'Industry import Growth'},inplace = True)
overall['Export concentration'] = overall['Industry export']/overall['Regional export']*100
grouped_data = overall.groupby(['Time', 'Area'])
result_df = pd.DataFrame()

for (time, area), group in grouped_data:
    # Find the row with maximum Export Concentration
    max_export_concentration = group['Export concentration'].max()
    max_export_rows = group[group['Export concentration'] == max_export_concentration]
    
    # Concatenate the rows to the result DataFrame
    result_df = pd.concat([result_df, max_export_rows])
result_df = result_df.drop(columns=result_df.columns[3:10])
result_df = result_df.drop(columns='Total GDP')
result_df['Industry'] = result_df['Industry'].astype(str).str[0]
result_df = result_df.rename(columns = {'Industry':'Dominant Industry'})
result_df.reset_index(inplace=True)
result_df = result_df.drop(columns='index')
result_df.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/Indices/regional_indexes.csv')
industries['Industry'] = industries['Industry'].astype(str).str[0]
industries = industries.drop(columns = industries.columns[5])
industries.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/Indices/industries_indexes.csv')

In [88]:
result_df

,Time,Area,Dominant Industry,Regional export,Regional import,Regional trade dependency,Regional import-Export imbalance,Regional export Growth,Regional import Growth,Export concentration
0,2015,MK01,G,1.691202e+10,3.347404e+10,61.084771,0.328702,0.000000,0.000000,13.346160
1,2015,MK02,Q,4.646912e+09,3.430935e+09,49.373111,-0.150532,0.000000,0.000000,16.991052
2,2015,MK04,C,3.348066e+09,1.207024e+09,55.997258,-0.470033,0.000000,0.000000,19.388457
3,2015,MK05,C,1.288965e+09,7.219806e+08,36.319109,-0.281949,0.000000,0.000000,16.681200
4,2015,MK06,C,4.161019e+09,3.159014e+09,41.505377,-0.136885,0.000000,0.000000,17.276955
...,...,...,...,...,...,...,...,...,...,...
109,2020,MK16,Q,1.892938e+09,2.028934e+09,148.775192,0.034676,0.103754,0.125935,19.992198
110,2020,MK17,Q,1.620991e+09,1.856990e+09,22.960239,0.067855,-0.289662,-0.152060,18.946149
111,2020,MK18,Q,3.950119e+08,1.210036e+08,20.525997,-0.531008,-0.102246,0.234731,18.777516
112,2020,MK19,Q,3.445914e+09,1.600960e+09,73.038080,-0.365564,-0.097478,-0.187762,19.179797


In [89]:
industries

,Time,Area,Industry,Export,Import,Industry trade dependency,Import-Export imbalance,Export Growth,Import Growth
0,2015,MK01,A,1.267644e+08,2.509054e+08,0.457862,0.328702,0.000000,0.000000
1,2015,MK01,B,7.414043e+06,1.467465e+07,0.026779,0.328702,0.000000,0.000000
2,2015,MK01,C,1.521114e+09,3.010749e+09,5.494135,0.328702,0.000000,0.000000
3,2015,MK01,D,1.516152e+08,3.000927e+08,0.547621,0.328702,0.000000,0.000000
4,2015,MK01,F,1.209391e+09,2.393754e+09,4.368218,0.328702,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
2047,2020,MK21,O,6.757047e+06,2.456386e+07,2.606209,0.568528,-0.217324,-0.256515
2048,2020,MK21,P,4.881392e+06,1.774530e+07,1.882764,0.568528,-0.190522,-0.231055
2049,2020,MK21,Q,1.273570e+07,4.629801e+07,4.912188,0.568528,-0.161811,-0.203783
2050,2020,MK21,R,4.337452e+06,1.576792e+07,1.672965,0.568528,-0.179286,-0.220383
